# Exercício 3 - Projeto 1 - Lógica Computacional

Grupo G20

Nome: Lara Catarina Vilaça Lopes ;
Número de Aluno: a108655

Nome: Fábio Mendes Castelhano ;
Número de Aluno: a105728

## 1. Abordagem utilizada para resolver o problema

1. Utilização da biblioteca OR-Tools

In [1]:
# Importação da biblioteca OR-Tools
from ortools.linear_solver import pywraplp

2. Criar a instância do solver

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# Definição dos parâmetros do problema, definidos pelo usuário
n = 5
m = 10 # Lembrar que m >= 2*n
q = 13 # Lembrar que q é primo e q >> m


3. Funções auxiliares para facilitar no momento da resolução do problema

Uso da biblioteca random

In [3]:
import random

Função geraMatriz, para gerar a matriz **H**, que tem **m** linhas e **n** colunas. Além disso, os elementos 
de **H** são inteiros gerados de forma aleatória, independente e uniforme num intervalo {0..q-1}

In [4]:
def geraMatriz(m, n, q):
    H = []
    for i in range(m):
        linha = []
        for j in range(n):
            linha.append(random.randint(0, q-1))
        H.append(linha)
    return H

H = geraMatriz(m, n, q)

Função imprimeMatriz, para facilitar a vizualização da matriz **H**

In [5]:
def imprimeMatriz(H):
    for linha in H:
        print(linha)
imprimeMatriz(H)        


[3, 0, 5, 5, 10]
[8, 8, 12, 2, 2]
[11, 0, 10, 5, 8]
[6, 11, 6, 11, 1]
[11, 9, 9, 11, 10]
[2, 11, 3, 4, 11]
[8, 12, 5, 10, 0]
[11, 3, 9, 8, 12]
[1, 9, 0, 4, 3]
[4, 12, 3, 8, 3]


Função geraVetor_e, que garante que o vetor **e** só terá 0's e 1's como elementos. A restrição
do vetor **e** ser diferente do vetor nulo ainda não foi levada em consideração.

In [6]:
def geraVetor_e(solver,m):
    return [solver.BoolVar(f'e[{i}]') for i in range(m)]

e = geraVetor_e(solver,m)

Função geraVetor_k, que garante que o vetor **k** terá apenas elementos inteiros e pertencentes
ao intervalo {0..m}

In [7]:
def geraVetor_k(solver,n,m):
    return [solver.IntVar(0, m, f'k[{j}]') for j in range(n)]

k = geraVetor_k(solver,n,m)

Restrições:

$$
\exists\, j < m \;\cdot\; e_j \neq 0
$$


Ou seja, a soma de todas as componentes do vetor **e** deve ser maior ou igual a 1. Adicionando-se ao solver, temos:

In [8]:
solver.Add(solver.Sum(e) >= 1) 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000029C6E9A37E0> >

$$
\forall\, i < n \;\cdot\; \sum_{j < m} e_j\, H_{j,i} \;=\; q\, k_i
$$


Adicionando-se ao solver, temos:

In [9]:
for i in range(n):
    solver.Add(sum(e[j] * H[j][i] for j in range(m)) - q * k[i] == 0)

Se existir o **e** que satisfaz as restrições, pretende-se determinar **e** que minimiza o 
número de componentes não nulas, ou seja, que faz com que a soma das componentes de **e** seja
mínima. Adicionando-se essa restrição ao solver, temos:

In [10]:
solver.Minimize(solver.Sum(e))

4. Função principal que resolve o problema

In [11]:
# Função principal que resolve o problema
def resolveProblema(n,m,q,H):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    e = geraVetor_e(solver,m)
    k = geraVetor_k(solver,n,m)

    solver.Add(solver.Sum(e) >= 1) 

    for i in range(n):
        solver.Add(sum(e[j] * H[j][i] for j in range(m)) - q * k[i] == 0)

    solver.Minimize(solver.Sum(e))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solução encontrada:")
        print("Matriz H:")
        imprimeMatriz(H)
        print("\nVetor e:", [int(var.solution_value()) for var in e])
        print("Vetor k:", [int(var.solution_value()) for var in k])
        print("Número de não-nulos em e:", int(sum(var.solution_value() for var in e)))
    else:
        print("Nenhuma solução encontrada:")
        print("Matriz H:")
        imprimeMatriz(H)

## 2. Exemplos práticos

### 1. Uma solução é encontrada

In [12]:
# Definição dos parâmetros do problema, definidos pelo usuário
n = 3
m = 6 # Lembrar que m >= 2*n
q = 31 # Lembrar que q é primo e q >> m
H = [
  [10, 20, 15],   
  [10, 2, 10],   
  [11, 5,  6],   
  [17, 24, 30],   
  [0,  7,  0],   
  [14, 4,  1],
]

resolveProblema(n,m,q,H)

Solução encontrada:
Matriz H:
[10, 20, 15]
[10, 2, 10]
[11, 5, 6]
[17, 24, 30]
[0, 7, 0]
[14, 4, 1]

Vetor e: [1, 1, 1, 1, 1, 1]
Vetor k: [2, 2, 2]
Número de não-nulos em e: 6


### 2.Uma solução não é encontrada

In [13]:
# Definição dos parâmetros do problema, definidos pelo usuário
n = 7
m = 14 # Lembrar que m >= 2*n
q = 71 # Lembrar que q é primo e q >> m
H = [[18, 44, 32, 55, 24, 43, 16],
[16, 68, 9, 8, 64, 20, 3],
[11, 39, 68, 16, 7, 58, 63],
[18, 8, 70, 11, 35, 36, 25],
[45, 26, 19, 21, 52, 12, 15],
[9, 65, 16, 38, 51, 34, 26],
[37, 7, 28, 20, 34, 23, 4],
[1, 11, 35, 13, 5, 64, 19],
[30, 52, 38, 33, 67, 11, 44],
[28, 31, 18, 17, 14, 5, 54],
[29, 63, 35, 20, 4, 10, 30],
[53, 27, 23, 29, 5, 56, 6],
[1, 57, 9, 31, 12, 41, 46],
[52, 47, 0, 1, 30, 24, 40]]
resolveProblema(n,m,q,H)

Nenhuma solução encontrada:
Matriz H:
[18, 44, 32, 55, 24, 43, 16]
[16, 68, 9, 8, 64, 20, 3]
[11, 39, 68, 16, 7, 58, 63]
[18, 8, 70, 11, 35, 36, 25]
[45, 26, 19, 21, 52, 12, 15]
[9, 65, 16, 38, 51, 34, 26]
[37, 7, 28, 20, 34, 23, 4]
[1, 11, 35, 13, 5, 64, 19]
[30, 52, 38, 33, 67, 11, 44]
[28, 31, 18, 17, 14, 5, 54]
[29, 63, 35, 20, 4, 10, 30]
[53, 27, 23, 29, 5, 56, 6]
[1, 57, 9, 31, 12, 41, 46]
[52, 47, 0, 1, 30, 24, 40]


### 3.Tentativas

In [14]:
# Definição dos parâmetros do problema, definidos pelo usuário
n = 10
m = 20 # Lembrar que m >= 2*n
q = 179 # Lembrar que q é primo e q >> m
H = geraMatriz(m, n, q)
resolveProblema(n,m,q,H)

Nenhuma solução encontrada:
Matriz H:
[12, 152, 94, 77, 23, 96, 44, 29, 85, 140]
[142, 16, 17, 19, 178, 50, 158, 87, 10, 35]
[81, 132, 148, 49, 128, 169, 6, 29, 169, 27]
[59, 35, 133, 88, 42, 70, 107, 22, 78, 77]
[0, 73, 106, 54, 168, 48, 34, 109, 134, 85]
[9, 89, 31, 118, 122, 73, 120, 96, 30, 5]
[97, 129, 27, 155, 157, 152, 7, 164, 58, 40]
[115, 78, 45, 91, 162, 145, 17, 66, 66, 33]
[131, 67, 168, 24, 48, 178, 72, 73, 84, 156]
[148, 32, 170, 130, 90, 25, 31, 28, 19, 28]
[176, 3, 83, 161, 92, 32, 133, 67, 0, 21]
[78, 165, 177, 41, 88, 98, 170, 92, 35, 77]
[72, 124, 93, 56, 152, 90, 127, 171, 85, 88]
[65, 73, 49, 177, 89, 40, 91, 144, 28, 33]
[7, 65, 142, 150, 42, 133, 112, 115, 174, 8]
[14, 121, 54, 156, 109, 99, 9, 97, 124, 69]
[90, 121, 173, 169, 130, 133, 29, 40, 5, 167]
[38, 168, 153, 20, 108, 5, 165, 129, 63, 69]
[59, 115, 154, 28, 161, 106, 78, 9, 112, 83]
[36, 176, 116, 53, 12, 160, 14, 39, 118, 164]
